In [504]:
from pymongo import MongoClient
import pymongo
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import re

In [505]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [506]:
#compMoney = db.companies.find({'$and':[{"total_money_raised":{"$regex":".*M.*"}},{"founded_year":{"$gte":2000}}]},{'name':1,'total_money_raised':1,'offices':1,'_id':0}).sort("total_money_raised",-1)
#df = pd.DataFrame(compMoney)

compMoney = db.companies.find({"total_money_raised":{"$regex":".*M.*"}},{'name':1,'total_money_raised':1,'offices':1,'_id':0}).sort("total_money_raised",-1)
df = pd.DataFrame(compMoney)

In [507]:
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)

In [508]:
cleanData = cleanData.drop(columns=["offices", "description", "address1", "address2", "zip_code"])
cleanData = cleanData.dropna()

In [509]:
atlanta = cleanData[cleanData['city'].str.contains("Atlanta")]
display(atlanta)
len(atlanta)

,name,total_money_raised,city,state_code,country_code,latitude,longitude
577,HowStuffWorks,$75M,Atlanta,GA,USA,37.090240,-95.712891
851,Zipcar,$60.7M,NW Atlanta,GA,USA,33.786263,-84.387794
949,Vertical Acuity,$6.08M,Atlanta,GA,USA,33.865682,-84.470944
1086,Google,$555M,Atlanta,GA,USA,33.781466,-84.387519
1442,EnerTech Environmental,$42M,Atlanta,GA,USA,33.772610,-84.350741
1505,Silverpop,$40M,Atlanta,GA,USA,33.882526,-84.465515
1508,Dayforce,$40M,Atlanta,GA,USA,33.887619,-84.710865
1554,Purewire,$4.75M,Atlanta,GA,USA,33.852384,-84.380568
1555,Purewire,$4.75M,Atlanta,GA,USA,33.852384,-84.380568
1824,Vitrue,$33M,Atlanta,GA,USA,33.756806,-84.392198


20

###### Vamos a intentar ahora hacer Web Scrapping para conseguir las coordenadas del aeropuerto de atlanta

In [510]:
url = 'https://es.wikipedia.org/wiki/Aeropuerto_Internacional_Hartsfield-Jackson'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [511]:
airportLat = soup.select('span.latitude')
airportLatitud = [a.text for a in airportLat]
airportLatitud[1]

'33.636666666667, '

In [512]:
airportLon = soup.select('span.longitude')
airportLongitud = [a.text for a in airportLon]
airportLongitud[1]

'-84.428055555556'